In [1]:
import pandas as pd
import os

import requests
import re
import time
import numpy as np
import subprocess
import random

from bs4 import BeautifulSoup

### extract categories

In [8]:
import googletrans

raw_data = []

url = "https://igroutka.net"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

for x in soup.find_all('a', class_='mn', href=True):
    if x['href'][0]!='/' or x['href'][-1]!='/':
        continue #skip (not game category)
    try:
        cat_name = x['title']
    except:
        continue
    
    cat_url = x['href']
#     print(cat_name, cat_url)
#     try:
#         cat_name_en = translator.translate(cat_name, dest='en')
#     except:
#         continue
    raw_data.append({'cat_url': cat_url, 
                     'cat_name': cat_name
#                      'cat_name_en': str(cat_name_en).split(',')[0]
                    })
df_cat = pd.DataFrame(raw_data)

In [9]:
df_cat

,cat_url,cat_name
0,/igry-dlya-detey/,Игры для детей
1,/igry-pazly/,Игры Пазлы
2,/igry-razvivayuschie/,Развивающие игры для детей
3,/dlya-malyshey/,Игры для малышей
4,/smeshariki/,Игры Смешарики
...,...,...
250,/igry-krokodilchik-svompi/,Игры Крокодильчик Свомпи
251,/igry-korabli/,Игры Корабли
252,/igry-vertolety/,Игры Вертолеты
253,/igry-sokrovischa/,Игры Сокровища


### extract game titles, url, play count, rating

In [32]:
games = []


for i, row in df_cat.iterrows():
    crawled = set()
    n_page = 1
    
    while True:
        game_curr = set()
        #get list of games for the category
        print("Scraping categoty", row['cat_name'], "https://igroutka.net" + row['cat_url'] + "page/" + str(n_page))
        response = requests.get("https://igroutka.net" + row['cat_url']  + "page/" + str(n_page))
        soup = BeautifulSoup(response.text, 'html.parser')


        game_list =  soup.find('div', id="gamelist")


        for x in game_list.find_all('a', class_='game-card'):
            if x.find('span', class_='game-rating icon-star star') == None:
                continue #skip if no rating info
            gameurl = x['href']
            gametitle = x.find('span', class_='name').text
            game_rating =  float(x.find('span', class_='game-rating icon-star star').text.replace(',','.'))
            play_count =  int(x.find('span', class_='game-rating icon-gamepad').text.replace(' ',''))
#             print(gameurl, gametitle, game_rating, play_count)
            games.append({'game': gametitle, 
                          'url':gameurl, 
                          'play_count':play_count, 
                          'rating':game_rating, 
                          'category':row['cat_name']})
            game_curr.add(gametitle)

        if game_curr - crawled == set():
            break
            
        n_page+=1
        crawled = crawled | game_curr
    

df_games = pd.DataFrame(games)
df_games.to_csv("./igroutka_game_info.csv")

Scraping categoty Игры для детей https://igroutka.net/igry-dlya-detey/page/1
Scraping categoty Игры для детей https://igroutka.net/igry-dlya-detey/page/2
Scraping categoty Игры для детей https://igroutka.net/igry-dlya-detey/page/3
Scraping categoty Игры для детей https://igroutka.net/igry-dlya-detey/page/4
Scraping categoty Игры для детей https://igroutka.net/igry-dlya-detey/page/5
Scraping categoty Игры для детей https://igroutka.net/igry-dlya-detey/page/6
Scraping categoty Игры для детей https://igroutka.net/igry-dlya-detey/page/7
Scraping categoty Игры Пазлы https://igroutka.net/igry-pazly/page/1
Scraping categoty Игры Пазлы https://igroutka.net/igry-pazly/page/2
Scraping categoty Игры Пазлы https://igroutka.net/igry-pazly/page/3
Scraping categoty Игры Пазлы https://igroutka.net/igry-pazly/page/4
Scraping categoty Развивающие игры для детей https://igroutka.net/igry-razvivayuschie/page/1
Scraping categoty Развивающие игры для детей https://igroutka.net/igry-razvivayuschie/page/2
Scr

Scraping categoty Игры Магазин https://igroutka.net/igry-magazin/page/2
Scraping categoty Игры Создай своего персонажа https://igroutka.net/igry-sozday-svoego-personazha/page/1
Scraping categoty Игры Создай своего персонажа https://igroutka.net/igry-sozday-svoego-personazha/page/2
Scraping categoty Игры Создай своего персонажа https://igroutka.net/igry-sozday-svoego-personazha/page/3
Scraping categoty Игры поцелуи https://igroutka.net/dlya-devochek/pocelui/page/1
Scraping categoty Игры поцелуи https://igroutka.net/dlya-devochek/pocelui/page/2
Scraping categoty Игры поцелуи https://igroutka.net/dlya-devochek/pocelui/page/3
Scraping categoty Игры Прически https://igroutka.net/dlya-devochek/pricheski/page/1
Scraping categoty Игры Прически https://igroutka.net/dlya-devochek/pricheski/page/2
Scraping categoty Игры Прически https://igroutka.net/dlya-devochek/pricheski/page/3
Scraping categoty Игры про собак https://igroutka.net/igry-pro-sobak/page/1
Scraping categoty Игры про собак https://i

Scraping categoty Игры Снайпер https://igroutka.net/igry-snayper/page/1
Scraping categoty Игры Снайпер https://igroutka.net/igry-snayper/page/2
Scraping categoty Игры Снайпер https://igroutka.net/igry-snayper/page/3
Scraping categoty Игры для мальчиков на двоих https://igroutka.net/na-dvoih-dlya-malchikov/page/1
Scraping categoty Игры для мальчиков на двоих https://igroutka.net/na-dvoih-dlya-malchikov/page/2
Scraping categoty Игры для мальчиков на двоих https://igroutka.net/na-dvoih-dlya-malchikov/page/3
Scraping categoty Игры для мальчиков на двоих https://igroutka.net/na-dvoih-dlya-malchikov/page/4
Scraping categoty Игры для мальчиков на двоих https://igroutka.net/na-dvoih-dlya-malchikov/page/5
Scraping categoty Игры для мальчиков на двоих https://igroutka.net/na-dvoih-dlya-malchikov/page/6
Scraping categoty Игры Паркур https://igroutka.net/igry-parkur/page/1
Scraping categoty Игры Паркур https://igroutka.net/igry-parkur/page/2
Scraping categoty Игры про войну https://igroutka.net/ig

Scraping categoty Игры Гонки на Выживание https://igroutka.net/igry-gonki-na-vyzhivanie/page/2
Scraping categoty Игры Гонки на Выживание https://igroutka.net/igry-gonki-na-vyzhivanie/page/3
Scraping categoty Игры Защита Королевства https://igroutka.net/igry-zaschita-korolevstva/page/1
Scraping categoty Игры Защита Королевства https://igroutka.net/igry-zaschita-korolevstva/page/2
Scraping categoty Игры Бомж Хобо https://igroutka.net/igry-bomzh-hobo/page/1
Scraping categoty Игры Бомж Хобо https://igroutka.net/igry-bomzh-hobo/page/2
Scraping categoty Игры Охота https://igroutka.net/igry-ohota/page/1
Scraping categoty Игры Охота https://igroutka.net/igry-ohota/page/2
Scraping categoty Игры Монстр-Траки https://igroutka.net/igry-gonki-na-monstr-trakah/page/1
Scraping categoty Игры Монстр-Траки https://igroutka.net/igry-gonki-na-monstr-trakah/page/2
Scraping categoty Игры Гонки на крутых машинах https://igroutka.net/igry-gonki-na-krutyh-masxyzh/page/1
Scraping categoty Игры Гонки на крутых м

Scraping categoty Игры на Скорость и Реакцию https://igroutka.net/igry-na-skorost-i-reakciyu/page/2
Scraping categoty Игры на Скорость и Реакцию https://igroutka.net/igry-na-skorost-i-reakciyu/page/3
Scraping categoty Флеш игры https://igroutka.net/flesh-igry/page/1
Scraping categoty Флеш игры https://igroutka.net/flesh-igry/page/2
Scraping categoty Флеш игры https://igroutka.net/flesh-igry/page/3
Scraping categoty Флеш игры https://igroutka.net/flesh-igry/page/4
Scraping categoty Флеш игры https://igroutka.net/flesh-igry/page/5
Scraping categoty Флеш игры https://igroutka.net/flesh-igry/page/6
Scraping categoty Флеш игры https://igroutka.net/flesh-igry/page/7
Scraping categoty Игры Динозавры https://igroutka.net/igry-dinozavry/page/1
Scraping categoty Игры Динозавры https://igroutka.net/igry-dinozavry/page/2
Scraping categoty Игры Динозавры https://igroutka.net/igry-dinozavry/page/3
Scraping categoty Игры Аниме https://igroutka.net/igry-anime/page/1
Scraping categoty Игры Аниме https:

Scraping categoty Игры Привет Сосед https://igroutka.net/igry-privet-sosed/page/2
Scraping categoty Игры по сети https://igroutka.net/igry-po-seti/page/1
Scraping categoty Игры по сети https://igroutka.net/igry-po-seti/page/2
Scraping categoty Игры по сети https://igroutka.net/igry-po-seti/page/3
Scraping categoty Игры Барашек Шон https://igroutka.net/igry-baranchik-shon/page/1
Scraping categoty Игры Барашек Шон https://igroutka.net/igry-baranchik-shon/page/2
Scraping categoty Игры Поиск Предметов https://igroutka.net/igry-poisk-predmetov/page/1
Scraping categoty Игры Поиск Предметов https://igroutka.net/igry-poisk-predmetov/page/2
Scraping categoty Игры Счастливая обезьянка https://igroutka.net/schastlivaya-obezyanka/page/1
Scraping categoty Игры Счастливая обезьянка https://igroutka.net/schastlivaya-obezyanka/page/2
Scraping categoty Игры Шашки https://igroutka.net/igry-shashki/page/1
Scraping categoty Игры Шашки https://igroutka.net/igry-shashki/page/2
Scraping categoty Игры Рыбка Е

Scraping categoty Игры Монстры https://igroutka.net/igry-monstry/page/2
Scraping categoty Игры Арканоид https://igroutka.net/igry-arkanoid/page/1
Scraping categoty Игры Арканоид https://igroutka.net/igry-arkanoid/page/2
Scraping categoty Игры для Телефона https://igroutka.net/igry-dlya-telefona/page/1
Scraping categoty Игры для Телефона https://igroutka.net/igry-dlya-telefona/page/2
Scraping categoty Игры для Телефона https://igroutka.net/igry-dlya-telefona/page/3
Scraping categoty Игры для Телефона https://igroutka.net/igry-dlya-telefona/page/4
Scraping categoty Игры для Телефона https://igroutka.net/igry-dlya-telefona/page/5
Scraping categoty Игры для Планшета https://igroutka.net/igry-dlya-plansheta/page/1
Scraping categoty Игры для Планшета https://igroutka.net/igry-dlya-plansheta/page/2
Scraping categoty Игры для Планшета https://igroutka.net/igry-dlya-plansheta/page/3
Scraping categoty Игры для Планшета https://igroutka.net/igry-dlya-plansheta/page/4
Scraping categoty Игры для Пл

In [33]:
df_games.drop_duplicates()

,game,url,play_count,rating,category
0,Игра Квест Пингвина,https://igroutka.net/igry-dlya-detey/32400-kve...,1146,5.0,Игры для детей
1,Игра Зоопаника,https://igroutka.net/dlya-devochek/zhivotnye/3...,623,4.0,Игры для детей
2,Игра Сабвей Бас Бегун 2020,https://igroutka.net/igry-subway-surfers/32381...,2712,4.3,Игры для детей
3,Игра Пузырьки Шутер: Конфеты,https://igroutka.net/igry-dlya-detey/32379-puz...,1082,3.8,Игры для детей
4,Игра Прыгающий Кенгуру,https://igroutka.net/igry-dlya-detey/32378-pry...,402,4.0,Игры для детей
...,...,...,...,...,...
21613,Игра Супер Магазин,https://igroutka.net/igry-biznes/22679-super-m...,17240,4.4,Игры Бизнес
21614,Игра Бизнес: Маленький Город,https://igroutka.net/igry-biznes/19467-malenki...,8188,4.4,Игры Бизнес
21615,Игра Мини Торговый Центр: Миллионер,https://igroutka.net/igry-biznes/18932-mini-to...,16643,4.3,Игры Бизнес
21616,Игра Управление Аэропортом,https://igroutka.net/igry-biznes/18837-upravle...,10775,4.4,Игры Бизнес


In [34]:
df_games.sort_values(by='play_count', ascending=False)

,game,url,play_count,rating,category
3760,Игра Пони Креатор v3: Создай Свою Пони 3,https://igroutka.net/pony/16349-igra-poni-krea...,8770982,4.6,Игры Создай своего персонажа
13672,Игра Пони Креатор v3: Создай Свою Пони 3,https://igroutka.net/pony/16349-igra-poni-krea...,8770982,4.6,Флеш игры
479,Игра Огонь и Вода 4 В хрустальном храме,https://igroutka.net/ogon-i-voda/2141-ogon-i-v...,7227523,4.5,Игры для детей
2774,Игра Огонь и Вода 4 В хрустальном храме,https://igroutka.net/ogon-i-voda/2141-ogon-i-v...,7227523,4.5,Игры для девочек на двоих
13737,Игра Огонь и Вода 4 В хрустальном храме,https://igroutka.net/ogon-i-voda/2141-ogon-i-v...,7227523,4.5,Флеш игры
...,...,...,...,...,...
1556,Игра Собери Картинку,https://igroutka.net/razvivayuschie-igry-dlya-...,97,4.0,Развивающие Игры Для Детей 3-4 года
13330,Игра Человек Авантюры,https://igroutka.net/raznye-igry/32005-chelove...,68,4.0,Игры на Скорость и Реакцию
18752,Игра Человек Авантюры,https://igroutka.net/raznye-igry/32005-chelove...,68,4.0,Разные игры
19447,Игра Домино Концентрация,https://igroutka.net/nastolnye/32297-domino-ko...,44,4.0,Настольные игры
